This code processes and visualizes the change in valid NDVI pixels over time for a defined study area.

Created on Tue July 23 2024

# @author: Pius N.Nwachukwu

In [ ]:
#  Masking and plotting the percentage of quality NDVI values across continents
# This code processes and visualizes the change in valid NDVI pixels over time for a defined study area.

In [ ]:
# Install the Earth Engine API
# !pip install earthengine-api

# Import the Earth Engine library and authenticate
import ee

# Authenticate
ee.Authenticate()

# Initialize the library with your project ID.
ee.Initialize(project='ee-')  # Replace 'your_project_id' with your actual project ID

# Define a study area: an example bounding box (adjust coordinates as needed)
study_area = ee.Geometry.Polygon([
      # [[19.85, -3.16], [19.85, -1.31], [22.39, -1.31], [22.39, -3.16], [19.85, -3.16]]  # "Africa" Ensure the polygon is closed by repeating the first coordinate
  # [[101.23, 29.02], [101.23, 29.54], [102.10, 29.54], [102.10, 29.02], [101.23, 29.02]] # "Asia"
  # [[122.37, -33.08], [122.37, -32.49], [122.85, -32.49], [122.85, -33.08], [122.37, -33.08]]  # "Aust_Ocean"
  # [[22.45, 45.12], [22.45, 45.47], [22.97, 45.47], [22.97, 45.12], [22.45, 45.12]]  # "Europe"
  # [[-121.68, 48.02], [-121.68, 49.01], [-119.80, 49.01], [-119.80, 48.02], [-121.68, 48.02]]  # "Nth_America"
  [[-60.70, -9.18], [-60.70, -7.56], [-58.42, -7.56], [-58.42, -9.18], [-60.70, -9.18]] # "Sth_America"
])

# Define the date range
start_date = '2000-02-01'
end_date = '2024-04-30'

# Load MODIS NDVI data
modis_ndvi = ee.ImageCollection('MODIS/061/MOD13A3') \
               .filterDate(start_date, end_date) \
               .filterBounds(study_area) \
               .select(['NDVI', 'SummaryQA'])

# Function to calculate the percentage of valid NDVI pixels
def calculate_valid_percentage(image):
    # Mask invalid NDVI values based on the quality reliability
    # Quality values 0, 1, 2, 3 indicate highest to lowest quality in MOD13A2
    quality_mask = image.select('SummaryQA').lte(2)
    valid_pixels = image.select('NDVI').updateMask(quality_mask).gt(-2000)

    total_pixels = image.select('NDVI').unmask().reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=study_area,
        scale=1000,
        maxPixels=1e9
    ).values().get(0)

    valid_pixel_count = valid_pixels.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=study_area,
        scale=1000,
        maxPixels=1e9
    ).values().get(0)

    percentage_valid = ee.Number(valid_pixel_count).divide(ee.Number(total_pixels)).multiply(100)
    return image.set('percentage_valid', percentage_valid).set('date', image.date().format('YYYY-MM-dd'))

# Apply the function to the image collection
valid_percentages = modis_ndvi.map(calculate_valid_percentage)

# Extract the dates and percentages separately
dates = valid_percentages.aggregate_array('date').getInfo()
percentages = valid_percentages.aggregate_array('percentage_valid').getInfo()

# Combine the results into a list of dictionaries
results = [{'date': date, 'percentage_valid': percentage} for date, percentage in zip(dates, percentages)]

# Print the results
print(results)

# Visualization
import matplotlib.pyplot as plt
import datetime
from matplotlib import dates as mdates

# Extract dates and valid percentages
dates = [entry['date'] for entry in results]
valid_percentages = [entry['percentage_valid'] for entry in results]

# Convert string dates to datetime objects
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(dates, valid_percentages, marker='o', linestyle='-')
plt.xlabel('Date')
plt.ylabel('Percentage of Valid NDVI Pixels (%)')
plt.title('Change in Valid NDVI Pixels Over South_America from IFL2020')
plt.grid(True)
plt.gca().xaxis.set_major_locator(mdates.YearLocator()) #Show major ticks
# Tilt x-axis labels
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.show()
